In [22]:
!pip install requests dotenv

In [23]:
import pandas as pd
import requests
import dotenv
import os
from datetime import datetime, timedelta

In [24]:
# Import API key from .env file
dotenv.load_dotenv()
aemet_api_key = os.getenv('aemet_api_key')

# Define the start and end dates
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 5, 31)

# Define the chunk size (6 months)
chunk_size = timedelta(days=180)

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through the date range in chunks of 6 months
current_date = start_date
while current_date <= end_date:
    chunk_end_date = min(current_date + chunk_size, end_date)

    # Format the dates as strings
    fechaini = current_date.strftime('%Y-%m-%dT00:00:00UTC')
    fechafin = chunk_end_date.strftime('%Y-%m-%dT00:00:00UTC')

    # Construct the URL
    url = f'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{fechaini}/fechafin/{fechafin}/estacion/0076'
    querystring = {"api_key": aemet_api_key}
    headers = {
        'cache-control': "no-cache"
    }

    # Make the request
    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()

    # Extract the data URL from the response
    data_url = data['datos']
    metadata_url = data['metadatos']

    # Make a request to the data URL
    data_response = requests.get(data_url)
    metadata_response = requests.get(metadata_url)

    # Parse the JSON data
    data_json = data_response.json()
    metadata_json = metadata_response.json()

    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(data_json)
    metadata_df = pd.DataFrame(metadata_json)

    # Append the dataframe to the list
    dataframes.append(df)

    # Move to the next chunk
    current_date = chunk_end_date + timedelta(days=1)

# Concatenate all dataframes into a single dataframe
final_df = pd.concat(dataframes, ignore_index=True)

# Check the size of the final dataframe
print(final_df.shape)

(1613, 25)


In [25]:
# Show the first 10 rows of the final dataframe
final_df.head(10)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,sol,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2020-01-01,0076,BARCELONA AEROPUERTO,BARCELONA,4,"9,2","0,0","4,9",05:52,"13,4",...,"4,9","1033,0",Varias,"1030,4",14,76,87,Varias,61,13:57
1,2020-01-02,0076,BARCELONA AEROPUERTO,BARCELONA,4,"9,4","0,0","4,1",Varias,"14,7",...,"1,5","1032,8",Varias,"1029,8",14,76,88,Varias,60,13:40
2,2020-01-03,0076,BARCELONA AEROPUERTO,BARCELONA,4,"7,8","0,0","4,5",07:30,"11,0",...,"1,6","1031,8",Varias,"1028,9",04,72,88,Varias,61,12:35
3,2020-01-04,0076,BARCELONA AEROPUERTO,BARCELONA,4,"9,4","0,0","3,9",06:38,"14,8",...,"5,3","1032,6",Varias,"1028,9",24,63,80,06:39,49,12:54
4,2020-01-05,0076,BARCELONA AEROPUERTO,BARCELONA,4,"10,0","0,0","4,9",07:22,"15,1",...,"5,5","1029,3",Varias,"1026,1",13,65,87,Varias,40,13:57
5,2020-01-06,0076,BARCELONA AEROPUERTO,BARCELONA,4,"10,0","0,0","4,1",07:14,"16,0",...,"5,4","1030,4",Varias,"1027,1",14,73,89,Varias,56,11:36
6,2020-01-07,0076,BARCELONA AEROPUERTO,BARCELONA,4,"10,0","0,0","4,6",07:30,"15,3",...,"5,4","1030,7",Varias,"1027,3",24,73,87,00:00,52,12:00
7,2020-01-08,0076,BARCELONA AEROPUERTO,BARCELONA,4,"10,7","0,0","4,7",07:34,"16,7",...,"5,3","1031,5",Varias,"1027,2",00,74,94,Varias,55,12:41
8,2020-01-09,0076,BARCELONA AEROPUERTO,BARCELONA,4,"10,2","4,9","5,7",07:52,"14,7",...,"5,5","1029,9",00,"1023,3",24,81,93,00:30,71,13:45
9,2020-01-10,0076,BARCELONA AEROPUERTO,BARCELONA,4,"9,6","16,2","6,8",22:25,"12,5",...,"0,0","1028,6",Varias,"1023,0",04,90,98,Varias,78,15:20


In [26]:
# Description of the fields

for i in range(len(metadata_df['campos'])):
    print(metadata_df['campos'][i]['id'] + ' : ' + metadata_df['campos'][i]['descripcion'])


fecha : fecha del dia (AAAA-MM-DD)
indicativo : indicativo climatológico
nombre : nombre (ubicación) de la estación
provincia : provincia de la estación
altitud : altitud de la estación en m sobre el nivel del mar
tmed : Temperatura media diaria
prec : Precipitación diaria de 07 a 07
tmin : Temperatura Mínima del día
horatmin : Hora y minuto de la temperatura mínima
tmax : Temperatura Máxima del día
horatmax : Hora y minuto de la temperatura máxima
dir : Dirección de la racha máxima
velmedia : Velocidad media del viento
racha : Racha máxima del viento
horaracha : Hora y minuto de la racha máxima
sol : Insolación
presmax : Presión máxima al nivel de referencia de la estación
horapresmax : Hora de la presión máxima (redondeada a la hora entera más próxima)
presmin : Presión mínima al nivel de referencia de la estación
horapresmin : Hora de la presión mínima (redondeada a la hora entera más próxima)
hrmedia : Humedad relativa media diaria
hrmax : Humedad relativa máxima diaria
horahrmax :

The required fields are fecha, tmed, prec, tmin, tmax, hrMedia, hrMax, hrMin, velmedia, sol

Insolacion es radiacion solar directa no ocultada por nubes o niebla

Ip significa precipitación inapreciable, es decir, cantidad inferior a 0.1 mm

In [29]:
useful_df = final_df[['fecha', 'tmed', 'prec', 'tmin', 'tmax', 'hrMedia', 'hrMax', 'hrMin', 'velmedia', 'sol']]
useful_df.head(10)

,fecha,tmed,prec,tmin,tmax,hrMedia,hrMax,hrMin,velmedia,sol
0,2020-01-01,"9,2","0,0","4,9","13,4",76,87,61,"2,5","4,9"
1,2020-01-02,"9,4","0,0","4,1","14,7",76,88,60,"4,7","1,5"
2,2020-01-03,"7,8","0,0","4,5","11,0",72,88,61,"5,8","1,6"
3,2020-01-04,"9,4","0,0","3,9","14,8",63,80,49,"5,0","5,3"
4,2020-01-05,"10,0","0,0","4,9","15,1",65,87,40,"5,0","5,5"
5,2020-01-06,"10,0","0,0","4,1","16,0",73,89,56,"2,2","5,4"
6,2020-01-07,"10,0","0,0","4,6","15,3",73,87,52,"3,9","5,4"
7,2020-01-08,"10,7","0,0","4,7","16,7",74,94,55,"2,5","5,3"
8,2020-01-09,"10,2","4,9","5,7","14,7",81,93,71,"4,7","5,5"
9,2020-01-10,"9,6","16,2","6,8","12,5",90,98,78,"3,3","0,0"


In [30]:
# Export the dataframe to a CSV file
useful_df.to_csv('weather_data_first_version.csv', index=False)